# This part bulids the bio age model 

## Use clinic outcome data and some related clinic data to calculate the adjusted CA by using risk scores

In [1]:
# importing module
from pandas import *
from statistics import mean
import numpy as np
from helper760_part2 import read_inputs

# Let's read all data and preprocess them
Clininc_Data,Outcome_Data,CT_Data = read_inputs()
n_samples = len(Clininc_Data[0])

# Average Life Expectancy in US in 2019 is 78.9 years. Therefore, we will use this as our average death age
# The general idea is that, if the Patient died, then we use ( 78.9 - death duration ) to compute adjust_CA
# Otherwise, we follow a rough estimation risk scores model to compute adjust_CA

ch_age = Clininc_Data[7]    # read patients's chronological age at CT
adjust_CA = []                # create a list to store adjust_CA of the patients
for i in range(n_samples):
    # if the patient has died, we can obtain a rough estimate of adjust_CA by subtract the death duration from 78.9, average life expectancy in the US
    if Outcome_Data[0][i] != 0:
        cur_adjust_CA = round(78.9 - float(Outcome_Data[0][i]/365.0),1)
        adjust_CA.append(cur_adjust_CA)
    else:
        diff = abs(ch_age[i] - 78.9)
        buff_range = 20
        risk_score = 0
        cur_adjust_CA = 0

        # if the age of the person is younger than 60, we consider them to be young, and assign light risk scores
        if ch_age[i] < 60:

            # BMI > 30
            if Clininc_Data[5][i] == 1:
                risk_score = risk_score + 0.06

            # FRS 10 year
            if Clininc_Data[10][i] > 0.2:
                risk_score = risk_score + 0.3
            elif Clininc_Data[10][i] > 0.1:
                risk_score = risk_score + 0.15
            
            # FRAX Fx Prob
            if Clininc_Data[11][i] > 30:
                risk_score = risk_score + 0.12
            elif Clininc_Data[11][i] > 10:
                risk_score = risk_score + 0.06
            
            # FRAX Fx Hip Prob
            if Clininc_Data[12][i] > 25:
                risk_score = risk_score + 0.06
            elif Clininc_Data[12][i] > 10:
                risk_score = risk_score + 0.03

            # Metabolism Syndrome
            if Clininc_Data[13][i] == 1:
                risk_score = risk_score + 0.06
            
            # Any cardiovascular problems CVD=stroke, Heart failure, MI=heart attack
            if Outcome_Data[1][i] == 1 or Outcome_Data[3][i] == 1 or Outcome_Data[5][i] == 1:
                risk_score = risk_score + 0.1

            # Diabetes
            if Outcome_Data[7][i] == 1:
                risk_score = risk_score + 0.05
            
            # Alzheimer
            if Outcome_Data[19][i] == 1:
                risk_score = risk_score + 0.05
            
            # Cancer
            if Outcome_Data[21][i] == 1:
                risk_score = risk_score + 0.2

            cur_adjust_CA = ch_age[i] + diff * risk_score

        elif ch_age[i] > 80: # old person, assign severe risk scores
            
            # BMI > 30
            if Clininc_Data[5][i] == 0:
                risk_score = risk_score + 0.06

            # FRS 10 year
            if Clininc_Data[10][i] < 0.1:
                risk_score = risk_score + 0.3
            elif Clininc_Data[10][i] < 0.2:
                risk_score = risk_score + 0.15
            
            # FRAX Fx Prob
            if Clininc_Data[11][i] < 10:
                risk_score = risk_score + 0.12
            elif Clininc_Data[11][i] < 30:
                risk_score = risk_score + 0.06
            
            # FRAX Fx Hip Prob
            if Clininc_Data[12][i] < 10:
                risk_score = risk_score + 0.06
            elif Clininc_Data[12][i] < 25:
                risk_score = risk_score + 0.03

            # Metabolism Syndrome
            if Clininc_Data[13][i] == 0:
                risk_score = risk_score + 0.06
            
            # Any cardiovascular problems CVD=stroke, Heart failure, MI=heart attack
            if Outcome_Data[1][i] == 0 and Outcome_Data[3][i] == 0 and Outcome_Data[5][i] == 0:
                risk_score = risk_score + 0.1

            # Diabetes
            if Outcome_Data[7][i] == 0:
                risk_score = risk_score + 0.05
            
            # Alzheimer
            if Outcome_Data[19][i] == 0:
                risk_score = risk_score + 0.05
            
            # Cancer
            if Outcome_Data[21][i] == 0:
                risk_score = risk_score + 0.2

            cur_adjust_CA = ch_age[i] - diff * risk_score

        # if the sample's age is close to US average life expectancy, assign intermediate risk scores
        else: 

            # BMI > 30
            if Clininc_Data[5][i] != 0:
                risk_score = risk_score + 0.03

            # FRS 10 year
            if Clininc_Data[10][i] > 0.3:
                risk_score = risk_score + 0.15
            
            # FRAX Fx Prob
            if Clininc_Data[11][i] > 30:
                risk_score = risk_score + 0.06

            
            # FRAX Fx Hip Prob
            if Clininc_Data[12][i] > 25:
                risk_score = risk_score + 0.03

            # Metabolism Syndrome
            if Clininc_Data[13][i] == 1:
                risk_score = risk_score + 0.03
            
            # Any cardiovascular problems CVD=stroke, Heart failure, MI=heart attack
            if Outcome_Data[1][i] == 0 and Outcome_Data[3][i] == 0 and Outcome_Data[5][i] == 0:
                risk_score = risk_score 
            else:
                risk_score = risk_score + 0.05

            # Diabetes
            if Outcome_Data[7][i] == 0:
                risk_score = risk_score 
            else:
                risk_score = risk_score + 0.025
            
            # Alzheimer
            if Outcome_Data[19][i] == 0:
                risk_score = risk_score 
            else:
                risk_score = risk_score + 0.025
            
            # Cancer
            if Outcome_Data[21][i] == 0:
                risk_score = risk_score 
            else:
                risk_score = risk_score + 0.1

            cur_adjust_CA = ch_age[i] + buff_range * risk_score


        cur_adjust_CA = round(cur_adjust_CA,1)
        adjust_CA.append(cur_adjust_CA)

## Use PCA approach to bulid the actual bio age model by using adjusted CA as our basis from above part

In [2]:
from helper760_part2 import read_inputs
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
import csv

# Analyze the number of cases and ratios that chronic age is less than bio age for all diease person
def analyze_disease(outcome,CA_original,BA,disease_ind,disease_name):
    diease = outcome[:,disease_ind]
    diease_ind = diease == 1
    print("--------------------------------------",disease_name,"---------------------------------")
    print("total case:",np.count_nonzero(diease == 1))
    diff = CA_original[diease_ind] - BA[diease_ind]
    print("CA - BA <= 0 case:",np.count_nonzero(diff <= 0))
    print("CA - BA <= 0 ratio:",np.count_nonzero(diff <= 0)/np.count_nonzero(diease == 1))
    return

Clininc_Data,Outcome_Data,CT_Data = read_inputs()
adjust_CA = np.array(adjust_CA)

ct = np.array(CT_Data)
ct = ct.T
ct = ct.astype(float)

clinic = np.array(Clininc_Data)
clinic = clinic.T
clinic = clinic.astype(float)

outcome = np.array(Outcome_Data)
outcome = outcome.T
outcome = outcome.astype(float)

CA_original = clinic[:,7] # Original chronic age

CA = adjust_CA # Use adjusted CA as our base CA to preform calculation

mean_ct = np.mean(ct,axis = 0)
std_ct = np.std(ct,axis = 0)

standardScalar = StandardScaler() 
standardScalar.fit(ct)
ct_std = standardScalar.transform(ct)

pca = PCA()
pca.fit(ct_std)

mean_CA = np.mean(CA,axis = 0)
std_CA = np.std(CA,axis = 0)

#Step 1: Compute the Biological scores (BScore).
#      Bscore = coefficients from PCA × Standardized CT data
B_score = np.zeros(ct.shape[0])
for i in range (ct.shape[0]):
    B_score_current = 0
    for j in range (ct.shape[1]):
        B_score_current =  B_score_current + pca.explained_variance_ratio_[j] * (ct[i][j] - mean_ct[j]) / std_ct[j] 
    B_score[i] =  B_score_current

#   Step 2: Tranfer Bscores to initial BAs.
#      BA’ = (BScore × SDCA ) + MeanCA 
BA = B_score * std_CA + mean_CA
BA_pad = np.column_stack([np.ones(len(BA)), BA]) # add constant column

# Step 3: Correct initial BAs to avoid overestimations or  underestimations.                 
#      BA = BA’ + (CAi –  MeanCA)(1 – b)
reg = LinearRegression().fit(BA_pad,CA)
b = reg.coef_[1]
Z = (CA - mean_CA) * (1-b)
BA = BA + Z
np.savetxt("bioAge.csv", BA, fmt='%f',delimiter=",")

# the predictions are:
# Idx 1 -> CVD
# Idx 3 -> Heart Failure
# Idx 5 -> MI DX
# Idx 7 -> Type 2 Diabetes
# Idx 9 -> Femoral Neck Fracture
# Idx 11 -> Unspec Femoral Fracture
# Idx 13 -> Forearm Fracture
# Idx 15 -> Humerus Fracture
# Idx 17 -> Pathologic Fracture
# Idx 19 -> Alzheimers
# Idx 21 -> Cancer (weather the patience has or doesn't have, regardless of 1 or 2 types)

diseaseNames = np.array(["CVD","Heart Failure","MI DX","Type 2 Diabetes","Femoral Neck Fracture"
                         ,"Unspec Femoral Fracture","Forearm Fracture","Humerus Fracture"
                         ,"Pathologic Fracture","Alzheimers","Cancer"])

# anaylze all disease cases
for i in range(diseaseNames.shape[0]):
    analyze_disease(outcome,CA_original,BA,2 * i + 1,diseaseNames[i])

any_diease = outcome[:,[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]] # all diease case
diease_sum = np.sum(any_diease,axis = 1)
diease_ind = diease_sum != 0
print("--------------------------------------all diease---------------------------------")
print("total case:",np.count_nonzero(diease_sum != 0))
diff = CA_original[diease_ind] - BA[diease_ind]
print("CA - BA <= 0 case:",np.count_nonzero(diff <= 0))
print("CA - BA <= 0 ratio:",np.count_nonzero(diff <= 0)/np.count_nonzero(diease_sum != 0))
print("--------------------------------------heath/unhealth comparison---------------------------------")
# Show all diease stat info
print("--------------------------------------unhealthSTAT---------------------------------")
CA_diease_mean = np.mean(CA_original[diease_ind])
BA_diease_mean = np.mean(BA[diease_ind])
CA_diease_std = np.std(CA_original[diease_ind])
BA_diease_std = np.std(BA[diease_ind])
print("CA_diease_mean",CA_diease_mean)
print("BA_diease_mean",BA_diease_mean)
print("CA_diease_std",CA_diease_std)
print("BA_diease_std",BA_diease_std)
# Show all health stat info
print("--------------------------------------healthSTAT---------------------------------")
health_ind = diease_sum == 0
CA_health_mean = np.mean(CA_original[health_ind])
BA_health_mean = np.mean(BA[health_ind])
CA_health_std = np.std(CA_original[health_ind])
BA_health_std = np.std(BA[health_ind])
print("CA_health_mean",CA_health_mean)
print("BA_health_mean",BA_health_mean)
print("CA_health_std",CA_health_std)
print("BA_health_std",BA_health_std)

-------------------------------------- CVD ---------------------------------
total case: 901
CA - BA <= 0 case: 644
CA - BA <= 0 ratio: 0.7147613762486127
-------------------------------------- Heart Failure ---------------------------------
total case: 643
CA - BA <= 0 case: 491
CA - BA <= 0 ratio: 0.7636080870917574
-------------------------------------- MI DX ---------------------------------
total case: 841
CA - BA <= 0 case: 659
CA - BA <= 0 ratio: 0.7835909631391201
-------------------------------------- Type 2 Diabetes ---------------------------------
total case: 2536
CA - BA <= 0 case: 1983
CA - BA <= 0 ratio: 0.7819400630914827
-------------------------------------- Femoral Neck Fracture ---------------------------------
total case: 126
CA - BA <= 0 case: 65
CA - BA <= 0 ratio: 0.5158730158730159
-------------------------------------- Unspec Femoral Fracture ---------------------------------
total case: 167
CA - BA <= 0 case: 95
CA - BA <= 0 ratio: 0.5688622754491018
--------